In [ ]:
# TotalSegmentator usage test

from totalsegmentator.python_api import totalsegmentator as ts

input_path = "ct.nii.gz"
output_path = "segmentations"

ts(input_path, output_path)


If you use this tool please cite: https://doi.org/10.48550/arXiv.2208.05868

Resampling...
  Resampled in 8.08s
Predicting part 1 of 5 ...
Predicting part 2 of 5 ...


In [ ]:
import SimpleITK as sitk
import os

In [13]:
# Process and format segmented files for visualization

import SimpleITK as sitk
import os

indir = "./segmentations/";
outdir = "./segmentations/mhd/";

main = sitk.ReadImage("ct.nii.gz", outputPixelType = sitk.sitkInt32)

if not os.path.exists(outdir) :
    os.makedirs(outdir)

for file in os.listdir(indir):
    if os.path.isfile(indir + file) and ".nii.gz" in file :
        img = sitk.ReadImage(indir + file, outputPixelType = sitk.sitkInt32)

        path = outdir + file.replace(".nii.gz", ".mhd")
        mask_filter = sitk.MultiplyImageFilter()
        res = mask_filter.Execute(main, img)

        minmax_filter = sitk.MinimumMaximumImageFilter()
        minmax_filter.Execute(img)
        
        if minmax_filter.GetMaximum() != 0 :
            sitk.WriteImage(res, path)

In [15]:
# More lightweight solution for just the layer mask

import SimpleITK as sitk
import os

indir = "./segmentations/";
outdir = "./segmentations/mhd/";

if not os.path.exists(outdir) :
    os.makedirs(outdir)
    
for file in os.listdir(indir):
    if os.path.isfile(indir + file) and ".nii.gz" in file :
        img = sitk.ReadImage(indir + file, outputPixelType = sitk.sitkUInt8)
        
        path = outdir + file.replace(".nii.gz", ".mhd")
        scale_filter = sitk.ShiftScaleImageFilter()
        scale_filter.SetScale(100)
        res = scale_filter.Execute(img)

        minmax_filter = sitk.MinimumMaximumImageFilter()
        minmax_filter.Execute(img)
        
        if minmax_filter.GetMaximum() != 0 :
            sitk.WriteImage(res, path)

In [64]:
import SimpleITK as sitk

img = sitk.ReadImage("ct.nii.gz")

sitk.WriteImage(img, "ct.mhd")

In [6]:
# Create visualizer

import platform
if platform.system() != 'Windows':
    import PySide2.QtSvg # Must import this before fast due to conflicting symbols
import fast # Must import FAST before rest of pyside2
from PySide2.QtWidgets import *
from PySide2.QtCore import Qt
from shiboken2 import wrapInstance
import random as rd
import functools
import keyboard
import os

segdir = "./segmentations/mhd/";

renderers = []
active = []

def import_segment(x) :
    print(x)
    
    importer = fast.ImageFileImporter.create(segdir + x)

    extraction = fast.SurfaceExtraction\
            .create(threshold=50)\
            .connect(importer)

    r = rd.uniform(0.5, 1);
    g = rd.uniform(0.5, 1);
    b = rd.uniform(0.5, 1);

    mesh = extraction.runAndGetOutputData()

    renderer = fast.TriangleRenderer\
            .create(color=fast.Color(r, g, b))\
            .connect(mesh)
    
    renderers.append(renderer)
    active.append(True)

    window.connect(renderer)
    
def toggle_segment_alpha(i):
    print(i)
    window.removeAllRenderers()
    window.connect(renderers[i])
    active[i] = True
    renderers[i].setOpacity(0, 1)
    
    changed = []
    
    for j in range(0, len(renderers)):
        if i != j and active[j]:
            window.connect(renderers[j])
            if not keyboard.is_pressed('q'):
                active[j] = False
                renderers[j].setOpacity(0, 0.05) 
                changed.append(j)
    
    for j in range(0, len(renderers)):
        if i != j and not active[j] and j not in changed:
            window.connect(renderers[j])
            renderers[j].setOpacity(0, 0.05)    
            
def reset_segment_alpha():
    for j in range(0, len(renderers)):
        renderers[j].setOpacity(0, 1)
        active[j] = True
            
    
def toggle_segment(i):
    return lambda: toggle_segment_alpha(i)

def export_mesh():
    return 0

# Create FAST Pipeline and window
window = fast.SimpleWindow3D.create(width=1024, height=512)\

# Get the underlying QtWidget of the FAST window and convert it to pyside2
mainWidget = wrapInstance(int(window.getWidget()), QWidget)

# Create GUI in Qt
layout = mainWidget.layout()
menuWidget = QWidget()
layout.addWidget(menuWidget)
menuLayout = QVBoxLayout()
menuWidget.setLayout(menuLayout)
menuLayout.setAlignment(Qt.AlignTop)
menuWidget.setFixedWidth(400)

# Create segment list
widget = QWidget()   

seglist = QVBoxLayout()
seglist.setAlignment(Qt.AlignTop)

scroll = QScrollArea()
scroll.setVerticalScrollBarPolicy(Qt.ScrollBarAlwaysOn)
scroll.setHorizontalScrollBarPolicy(Qt.ScrollBarAlwaysOff)
scroll.setWidgetResizable(True)

files = os.listdir(segdir)

idx = 0

for i in range(0, len(files)) :
    if os.path.isfile(segdir + files[i]) and ".mhd" in files[i] :
        elem_button = QPushButton(files[i][:-4])
        elem_button.clicked.connect(toggle_segment(idx))
        import_segment(files[i])
        seglist.addWidget(elem_button)
        idx += 1
        
widget.setLayout(seglist)
scroll.setWidget(widget)
        
reset = QPushButton("Reset view")
reset.clicked.connect(lambda: reset_segment_alpha())
    
menuLayout.addWidget(reset)
menuLayout.addWidget(scroll)

reset = QPushButton(files[i][:-4])
        
# Run everything!
window.run()

adrenal_gland_left.mhd
adrenal_gland_right.mhd
aorta.mhd
autochthon_left.mhd
autochthon_right.mhd
clavicula_left.mhd
clavicula_right.mhd
colon.mhd
duodenum.mhd
esophagus.mhd
femur_left.mhd
femur_right.mhd
gallbladder.mhd
gluteus_maximus_left.mhd
gluteus_maximus_right.mhd
gluteus_medius_left.mhd
gluteus_medius_right.mhd
gluteus_minimus_left.mhd
gluteus_minimus_right.mhd
heart_atrium_left.mhd
heart_atrium_right.mhd
heart_myocardium.mhd
heart_ventricle_left.mhd
heart_ventricle_right.mhd
hip_left.mhd
hip_right.mhd
humerus_left.mhd
humerus_right.mhd
iliac_artery_left.mhd
iliac_artery_right.mhd
iliac_vena_left.mhd
iliac_vena_right.mhd
iliopsoas_left.mhd
iliopsoas_right.mhd
inferior_vena_cava.mhd
kidney_left.mhd
kidney_right.mhd
liver.mhd
lung_lower_lobe_left.mhd
lung_lower_lobe_right.mhd
lung_middle_lobe_right.mhd
lung_upper_lobe_left.mhd
lung_upper_lobe_right.mhd
pancreas.mhd
portal_vein_and_splenic_vein.mhd
pulmonary_artery.mhd
rib_left_1.mhd
rib_left_10.mhd
rib_left_11.mhd
rib_left_12.mhd

In [6]:
# https://github.com/pthom/imgui_bundle/blob/main/bindings/imgui_bundle/demos_python/notebooks/demo_notebook.ipynb

In [2]:
dir(fast.Mesh)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__swig_destroy__',
 '__weakref__',
 'accessFinished',
 'clearLastFrame',
 'create',
 'deleteMetadata',
 'fromDataObject',
 'getBoundingBox',
 'getCreationTimestamp',
 'getFrameData',
 'getLastFrame',
 'getMeshAccess',
 'getMetadata',
 'getNameOfClass',
 'getNrOfLines',
 'getNrOfTriangles',
 'getNrOfVertices',
 'getOpenCLAccess',
 'getReporter',
 'getSceneGraphNode',
 'getStaticNameOfClass',
 'getTimestamp',
 'getTransform',
 'getTransformedBoundingBox',
 'getVertexBufferObjectAccess',
 'hasFrameData',
 'isLastFrame',
 'removeLastFrame',
 'setBoundingBox',
 'setCreationTimestamp',
 'setFrameData',
 'setLastFrame',
 'setMetadata',
 'setTr

In [4]:
from stl import mesh as stmesh
import math
import numpy as np
import fast

indir = "./segmentations/mhd/";
outdir = "./meshes/";
# Spine
#files = [\
#    "vertebrae_L1",\
#    "vertebrae_L2",\
#    "vertebrae_L3",\
#    "vertebrae_L4",\
#    "vertebrae_L5",\
#    "vertebrae_T1",\
#    "vertebrae_T10",\
#    "vertebrae_T11",\
#    "vertebrae_T12",\
#    "vertebrae_T2",\
#    "vertebrae_T3",\
#    "vertebrae_T4",\
#    "vertebrae_T5",\
#    "vertebrae_T6",\
#    "vertebrae_T7",\
#    "vertebrae_T8",\
#    "vertebrae_T9",\
#    "sacrum"];

# Heart
files = [\
    "heart_atrium_left",\
    "heart_atrium_right",\
    "heart_myocardium",\
    "heart_ventricle_left",\
    "heart_ventricle_right"];

verts = []
faces = []

offset = 0

for file in files:
    importer = fast.ImageFileImporter.create(indir + file + ".mhd")
    smoothing = fast.GaussianSmoothing.create(stdDev=2.0).connect(importer)
    extraction = fast.SurfaceExtraction.create(threshold=50).connect(smoothing)
    mesh = extraction.runAndGetOutputData()
    access = mesh.getMeshAccess(fast.ACCESS_READ)

    for vertex in access.getVertices():
        verts.append(vertex.getPosition().reshape(1, 3))

    for face in access.getTriangles():
        faces.append([offset + face.getEndpoint1(), offset + face.getEndpoint2(), offset + face.getEndpoint3()])
        
    offset = len(verts)
    
    print(file)
    
verts = np.array(verts)
faces = np.array(faces)
    
model = stmesh.Mesh(np.zeros(faces.shape[0], dtype=stmesh.Mesh.dtype))
for i, f in enumerate(faces):
    for j in range(3):
        model.vectors[i][j] = verts[f[j],:]
    
model.save(outdir + "output.stl")

heart_atrium_left
heart_atrium_right
heart_myocardium
heart_ventricle_left
heart_ventricle_right
